In [2]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from autogen import AssistantAgent
from autogen.agentchat import groupchat
import os

In [3]:
# MongoDB connection details
MONGO_URI = os.environ["MONGODB_URI"]  # Replace with your MongoDB URI
DB_NAME = "vector_mflix"
COLLECTION_NAME = "movies_with_vectors"

In [4]:
# Initialize MongoDB client
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

In [5]:
# Load pre-trained embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [6]:
def embed_plot(plot_text):
    """Generate vector embedding for the input plot."""
    return embedding_model.encode(plot_text).tolist()

In [7]:
def find_closest_movies(plot_embedding, top_k=5):
    """Find the top-k closest movies."""
    pipeline = [
        {
            "$addFields": {
                "embedding_distance": {
                    "$let": {
                        "vars": {"plot_emb": "$plot_embedding"},
                        "in": {
                            "$sqrt": {
                                "$sum": [
                                    {"$pow": [{"$subtract": [e, {"$arrayElemAt": ["$$plot_emb", i]}]}, 2]}
                                    for i, e in enumerate(plot_embedding)
                                ]
                            }
                        },
                    }
                }
            }
        },
        {"$sort": {"embedding_distance": 1}},
        {"$limit": top_k}
    ]
    return list(collection.aggregate(pipeline))

In [8]:
# Define agents
class MovieFinderAgent(AssistantAgent):
    def __init__(self):
        super().__init__(name="Movie Finder", description="Finds movies based on plot descriptions.")
    
    def respond(self, input_text):
        """Find potential matches for the input plot."""
        plot_embedding = embed_plot(input_text)
        results = find_closest_movies(plot_embedding, top_k=5)
        if not results:
            return "No movies found."
        return results

In [9]:
class MovieValidatorAgent(AssistantAgent):
    def __init__(self):
        super().__init__(name="Movie Validator", description="Validates the identified movie with the user.")

    def respond(self, candidate_movie):
        """Present the candidate movie to the user for validation."""
        response = f"Is this the movie you're looking for?\nTitle: {candidate_movie['title']} ({candidate_movie['year']})\nGenres: {', '.join(candidate_movie['genres'])}\nPlot: {candidate_movie['plot']}\n"
        return response

In [11]:
# Chat Orchestration
def movie_chat():
    print("Welcome to the Movie Finder Chat!")
    user_input = input("Describe the movie plot: ")

    # Create agents
    finder_agent = MovieFinderAgent()
    validator_agent = MovieValidatorAgent()

    # Initialize group chat
    chat = groupchat([finder_agent, validator_agent], name="Movie Finder Group Chat")

    # Finder agent finds matches
    matches = finder_agent.respond(user_input)
    if isinstance(matches, str):  # No movies found
        print(matches)
        return

    # Validate each movie
    for candidate in matches:
        validator_response = validator_agent.respond(candidate)
        print(validator_response)
        
        user_feedback = input("Is this the correct movie? (yes/no): ").strip().lower()
        if user_feedback == "yes":
            print(f"Great! Movie identified: {candidate['title']} ({candidate['year']})")
            return

    print("Sorry, we couldn't identify the movie. Please try again with more details.")

In [12]:
# Run the chat
if __name__ == "__main__":
    movie_chat()


Welcome to the Movie Finder Chat!


TypeError: 'module' object is not callable